In [2]:
import sys
sys.path.append('../../framework')
sys.path.append('../../application')

from NetworkClass import Network

In [9]:
model_dict = {
        "network": {
            'input_layer': {
                "units": 784,

            },
            'hidden_layer': [{
                "units": 168,
                "activation": "relu",
                "type": "Linear"
            },
                {
                    "units": 168,
                    "activation": "relu",
                    "type": "Linear"

            },
                {
                    "units": 168,
                    "activation": "relu",
                    "type": "Linear"

            }
            ],
            'output_layer': {
                "units": 10,
                "activation": "softmax",
                "type": "Linear"
            }
        }
    }
model = Network(model_dict)
model

Network(
  (input_layer): Linear(in_features=784, out_features=168, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=168, out_features=168, bias=True)
    (1): Linear(in_features=168, out_features=168, bias=True)
  )
  (output_layer): Linear(in_features=168, out_features=10, bias=True)
)

In [ ]:
import logging
import string
import random
import os
import torch
import torchvision
from sklearn.model_selection import KFold

from Experiment import Experiment
from train_utils import ReshapeTransform


def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

logging.basicConfig(level=logging.INFO)

if not(os.path.isdir('models')):
  os.mkdir('models')

params_dict = {
  "batch_size_train": 100,
  "learning_rate": 0.01,
  "batch_size_test": 1000,
  "n_epochs": 200
}

seed = 42
uid = randomString(stringLength=6)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  torch.cuda.manual_seed(seed)
else:
  torch.manual_seed(seed)


params_dict["model"] = model_dict

train_dataset = torchvision.datasets.FashionMNIST('../data/', train=True, download=True,
                                                                  transform=torchvision.transforms.Compose([
                                                                      torchvision.transforms.ToTensor(),
                                                                      ReshapeTransform(
                                                                          (-1,))
                                                                  ]))

test_dataset = torchvision.datasets.FashionMNIST('../data/', train=False, download=True,
                                                                 transform=torchvision.transforms.Compose([
                                                                     torchvision.transforms.ToTensor(),
                                                                     ReshapeTransform(
                                                                         (-1,))
                                                                 ]))

dataset = torch.utils.data.ConcatDataset(
                    [train_dataset, test_dataset])

kf = KFold(n_splits=5, shuffle=True, random_state=seed)
for i_fold, (train_index, test_index) in enumerate(kf.split(dataset)):
    print("Fold: {}".format(i_fold+1))
    # new fold - network from scratch
    experiment = Experiment(device)
    model = Network(model_dict)
    params_dict["fold"] = i_fold+1
    # set the dataloaders for the fold
    train = torch.utils.data.Subset(dataset, train_index)
    test = torch.utils.data.Subset(dataset, test_index)
    train_loader = torch.utils.data.DataLoader(
        train, batch_size=params_dict["batch_size_train"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        test, batch_size=params_dict["batch_size_test"], shuffle=True)
    # set up the experiment
    experiment.set_metadata(params_dict)
    experiment.set_network(model_dict)
    experiment.set_loaders(train_loader, test_loader)
    experiment.set_loss(torch.nn.CrossEntropyLoss())

    # training loop
    for idx, epoch in enumerate(range(params_dict["n_epochs"])):
        print("Epoch: {}".format(epoch))
        epoch_vals = experiment.train_epoch(epoch)
        logging.info(epoch_vals)
        logging.info(experiment.network)
        experiment.save_weights({
            'epoch': epoch,
            'state_dict': experiment.network.state_dict(),
            'train_acc': experiment.tacc,
            'val_acc': experiment.acc,
            'train_loss': experiment.trainLoss,
            'val_loss': experiment.testLoss,
            'optimizer': experiment.optimizer.state_dict(),
            'traint': experiment.traint,
            'traini': experiment.traini,
            'params': experiment.params_dict
        }, 'models/{}_{}.pth.tar'.format(uid, epoch,))


Fold: 1
Epoch: 0


1026-INFO-{'training_time': 8.117594718933105, 'inference_time': 0.010701383863176619, 'model_parameters': 190354, 'training_acc': tensor(10.2125), 'test_acc': tensor(11.3714)}
1026-INFO-Network(
  (input_layer): Linear(in_features=784, out_features=168, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=168, out_features=168, bias=True)
    (1): Linear(in_features=168, out_features=168, bias=True)
  )
  (output_layer): Linear(in_features=168, out_features=10, bias=True)
)


Epoch: 1


1026-INFO-{'training_time': 7.437199115753174, 'inference_time': 0.009059395108904158, 'model_parameters': 190354, 'training_acc': tensor(12.4911), 'test_acc': tensor(15.8000)}
1026-INFO-Network(
  (input_layer): Linear(in_features=784, out_features=168, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=168, out_features=168, bias=True)
    (1): Linear(in_features=168, out_features=168, bias=True)
  )
  (output_layer): Linear(in_features=168, out_features=10, bias=True)
)


Epoch: 2


...

Epoch: 19


1026-INFO-{'training_time': 6.9388768672943115, 'inference_time': 0.009324380329677038, 'model_parameters': 190354, 'training_acc': tensor(30.1054), 'test_acc': tensor(28.6643)}
1026-INFO-Network(
  (input_layer): Linear(in_features=784, out_features=168, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=168, out_features=168, bias=True)
    (1): Linear(in_features=168, out_features=168, bias=True)
  )
  (output_layer): Linear(in_features=168, out_features=10, bias=True)
)


Epoch: 20


1026-INFO-{'training_time': 7.671156883239746, 'inference_time': 0.00856477873665946, 'model_parameters': 190354, 'training_acc': tensor(27.6304), 'test_acc': tensor(24.9643)}
1026-INFO-Network(
  (input_layer): Linear(in_features=784, out_features=168, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=168, out_features=168, bias=True)
    (1): Linear(in_features=168, out_features=168, bias=True)
  )
  (output_layer): Linear(in_features=168, out_features=10, bias=True)
)


Epoch: 21
